In [3]:
from logging import exception
import sqlalchemy as sa
import urllib
import pandas as pd
import numpy as np
parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                    "SERVER=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
                                    "DATABASE=DW_Central;"
                                    "UID=sa;"
                                    "PWD=DWpln21")

In [96]:
df_original = pd.read_excel("y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo Mayo.xlsb", sheet_name='Export', skipfooter=3)
df = df_original.copy(deep=True)
df = pd.DataFrame(df_original)
df['POC_ID'] = df['POC_ID'].astype('string')
# df.dtypes
# df.sample(10)

In [97]:
df['COD_CLIENTE'] = df.POC_ID.str.extract('(0{2,}\d+)')
df['COD_DISTRI'] = df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)')
df = df[df.COD_DISTRI.isin(['704250','328660','369117','369846','445625'])]
df['COD_DISTRI'] = df['COD_DISTRI'].astype('Int64')
df['COD_CLIENTE'] = df['COD_CLIENTE'].astype('Int64')
df.drop(columns=['Ordenes inf. Upsell','POC_ID','Ordenes inf.Quick Order'], inplace=True)
# df.nunique()
# df.sample(10)

In [98]:
df

,Ordenes Influenciadas,Ordenes totales,COD_CLIENTE,COD_DISTRI
7645,2,2,16,704250
7646,0,3,17,704250
7647,0,2,19,704250
7648,0,4,24,704250
7649,2,4,31,704250
...,...,...,...,...
102212,0,4,987625,445625
102213,5,5,987633,445625
102214,0,3,987638,445625
102215,2,4,999111,445625


In [ ]:
# df_m = df.loc[0:4,['POC_ID', 'Ordenes Influenciadas', 'Ordenes totales']] 
# df_m

In [203]:
df['Periodo'] = pd.to_datetime('1/6/2022')
df.convert_dtypes()
# df['POC_ID'].astype('Int64')
# df.dtypes

POC_ID                              int64
Ordenes Influenciadas               int64
Ordenes inf. Upsell                 int64
Ordenes inf.Quick Order             int64
Ordenes totales                     int64
Periodo                    datetime64[ns]
dtype: object

In [204]:
# conditionlist = [
#     (df['POC_ID']>=7042500000000) & (df['POC_ID']<7042600000000),
#     (df['POC_ID']>=32866000000000) & (df['POC_ID']<32866100000000),
#     (df['POC_ID']>=36911700000000) & (df['POC_ID']<36911800000000),
#     (df['POC_ID']>=36984600000000) & (df['POC_ID']<36984700000000),
#     (df['POC_ID']>=44562500000000) & (df['POC_ID']<44562600000000)]
# choicelist = ['70425', '328660', '369117', '369846', '445625']
# df['DISTRI'] = np.select(conditionlist, choicelist, default = 'OTRO DISTRI')
# df.sample

In [206]:
try: 
    df.drop('Ordenes inf. Upsell', axis=1, inplace=True)
except: 
    pass 
# df.groupby(['DISTRI', 'Periodo']).sum()
# df


In [207]:
# df.drop(columns='DISTRI'=='OTRO DISTRI',axis=0)
# df
df = df[df['DISTRI']!='OTRO DISTRI']
# df

In [214]:
def run():
    try:
        engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
        print("Conect succesfull")
    except Exception:
        print("Errora in DataBase conection")
    try:    
        df.to_sql('Algoritmo', con = engine, if_exists= 'append', index=False)
        print("carga exitosa")
    except Exception:
        print("Error fatal irrecuperable")

run()


Errora in DataBase conection
Error fatal irrecuperable
